# Getting Started Replaying Old Data

- Work under a separate branch with git (i.e. git checkout -b my-new-branch)

- Copy this ipynb file to a folder within your Projects folder - better not work within this (Projects_Templates/2_ReplayOldDataAsNF) folder. Come up with a nice name for the new folder.

- Then enter your new folder within Projects (maybe 3_New_Assignment_Fancy_Analysis), and use the code in here for for your purposes as you see fit

- you might wish to grab some data from Leipzig here: ldrive, under **Lab_MichaelB/Johan/nf/rawdata/ReplayDataTest**


- the main gist of re-playing existing data is:
    - that your first load everything into a big matrix with mne
    - then initialize a specific **amp** (i.e., the "replayamp")
    - then do exactly the same as normal, with a while True loop with calls to amp.get_data()
    
- you may wish to put all of the import statements at the top

- if you get a module not found error, add the mushu folder to the python path. A good analogy is the matlab's 'addpath' utility.


For example:
```python
sys.path.append('/home/nfcontrol/nf/nf-rtime/mushu/')      # this is to talk with the amplifer
sys.path.append('/home/nfcontrol/nf/nf-rtime/wyrm/') 
```


Not all of this code is useful! So remove the stuff you don't need, f.e. matplogib qt5, path appends, etc etc.

    
    




In [14]:
def plot_psd_pb(filename, metafile, markerfile, *args, **kwargs):
    ''' 
    This function plots the PSD (in dB) of the frequency range of your choice.
    This function is to be used after EEG acquisition. The PSD compares two events 
    if you so choose (such as eyes open-eyes closed) or you could plot just one 
    event and look at the topographical distribution at a particular frequency on the scalp.
    
    Uses MNE-Python.
        
        
    Sample usage: plot_psd_pb('file.eeg', 'file.meta', 'file.marker', 
                                frequency_range=[9, 12], n_samples=500000, 
                                replay, n_fft=256, montage='biosemi64',
                                resample_freq=1000)
    
    NOTE: At present, this only works for comparing eyes open and eyes closed with the marker codes
    from the EEG acquisition using the EEG-fMRI Localiser.
    '''
    
    import struct
    import json
    import time
    import matplotlib
    %matplotlib qt5
    import numpy as np
    import matplotlib.pyplot as plt
    import mne
    from mne.time_frequency import psd_welch
    import re
    
    import sys
    sys.path.append('C:/Users/nikitasK/repos/nf-rtime-master/mushu')
    sys.path.append('C:/Users/nikitasK/repos/nf-rtime-master/mushu/libmushu')
    
    import libmushu
    amp = libmushu.get_amp('replayamp')
    
    f = open(filename, 'r')
    raw = np.fromfile(f, dtype=np.float32)
    raw = raw.reshape(round(len(raw)/64),64).transpose()
    
    if 'n_samples' not in kwargs.keys():
        n_samples = 500000
    else:
        if kwargs['n_samples'] > len(raw):
            raise ValueError('Number of samples cannot be greater than the number of samples in the EEG file (%d)' % len(raw))
        n_samples = kwargs['n_samples']
    
    if 'montage' not in kwargs.keys():
        montage = mne.channels.read_montage(kind='biosemi64')
        
    else:
        montage = mne.channels.read_montage(kind=kwargs['montage'])
        
    ch = montage.ch_names[:64]
    fs = len(raw[0])
    m = raw[0:n_samples]
    m = np.array(m.transpose())
    
    ## replay file
    if 'replay' in args:
        
        amp.configure(m, (), ch, fs, realtime=True, blocksize_samples=20)
        amp.start()
        alld=[]
        allm=[]
        starttime=time.time()
        newtime=starttime
        i=0;
        while time.time() - starttime < n_samples/fs:

            while time.time() - newtime < 0.5:
                pass
            else:
                data, marker = amp.get_data()
        
                alld.append(data)
                for m in marker:
                    allm.append(marker)
                print(marker)
                
        
        

            print('%d' % i, end='', flush=True)
            i+=1
            newtime+=0.5
            
        amp.stop()
        m=np.concatenate(alld)
            
    if 'frequency_range' in kwargs.keys():
        if len(kwargs['frequency_range']) is not 2:
            raise ValueError('Frequency range length is %d, needs to be 2.' % len(frequency_range))
        frequency_range = kwargs['frequency_range']
    else:
        frequency_range = [2,25]
        
    fmin, fmax = frequency_range     
            
    
            
            
    ## get markers and meta
    fhj=open(metafile)
    meta=json.load(fhj)
    
    info = mne.create_info(
    ch_names = ch,
    ch_types = ['eeg' for i in range(64)],
    sfreq=meta['Sampling Frequency'],
    montage = montage
    )
    
    with open(markerfile) as file:
        content = file.readlines()
        
    content = [x.strip() for x in content] 
    
    ## get data
    rt = np.transpose(m)    
    rt_raw = mne.io.RawArray(rt, info)

    # create the marker matrix
    ev_arr=[]
    for i, item in enumerate(content):
        out=re.split("[\s|S|T]+", item)
        if 'Sync Off' in item:
            sample = int(float(out[0])/1000*meta['Sampling Frequency'])
            code = 250
        else:
            sample = int(float(out[0])/1000*meta['Sampling Frequency'])
            code = int(out[1])

        ev_arr.append([sample, 0, code])
    
    ev_arr = ev_arr[:4]
    
    # create marker channel for MNE python:
    if ev_arr:
        stim_info = mne.create_info(['STI'], rt_raw.info['sfreq'], ['stim'])
        stim_data = np.zeros((1, len(rt_raw.times)))
        stim_raw = mne.io.RawArray(stim_data, stim_info)
        rt_raw.add_channels([stim_raw], force_update_info=True)

        # create the marker matrix:
        rt_raw.add_events(ev_arr)
        
    events = mne.find_events(rt_raw)
    
    if 'resample_freq' in kwargs.keys():
        print('Resampling to %f, please be patient...' % kwargs['resample_freq'])
        rt_raw.resample(kwargs['resample_freq'], npad='auto')
        
    mne.set_eeg_reference(rt_raw)
        
        
    ## eyes open
    raw_eo = rt_raw.copy().crop(events[0][0]/info['sfreq']-1, events[1][0]/info['sfreq']+1)

    ## eyes closed
    raw_ec = rt_raw.copy().crop(events[2][0]/info['sfreq']-1, events[3][0]/info['sfreq']+1)
    

    # do psds
    if 'n_fft' not in kwargs.keys():
        n_fft = 16384
        
    else:
        n_fft = kwargs['n_fft']
    psd_eo, freqs_eo = psd_welch(raw_eo, fmin=fmin, fmax=fmax, n_fft=n_fft)
    psd_ec, freqs_ec = psd_welch(raw_ec, fmin=fmin, fmax=fmax, n_fft=n_fft)
    
    log_psd_eo = 10 * np.log10(psd_eo)
    log_psd_ec = 10 * np.log10(psd_ec)
    fig, ax = plt.subplots(1, 2)
    psds_mean_eo = log_psd_eo.mean(0)
    psds_mean_ec = log_psd_ec.mean(0)
    psds_diff = log_psd_ec - log_psd_eo
    psds_diff_mean = psds_diff.mean(0)
    x=[freqs_eo[int(np.where(psds_diff_mean == np.max(psds_diff_mean))[0][0]-np.std(psds_diff_mean.round()))], 
       freqs_eo[int(np.where(psds_diff_mean == np.max(psds_diff_mean))[0][0]+np.std(psds_diff_mean).round())]]

    ## plot psds
    ax[0].plot(freqs_eo, psds_mean_eo, color='r', label='Eyes open')
    ax[0].plot(freqs_ec, psds_mean_ec, color='b', label='Eyes closed')
    ax[0].plot(freqs_eo, psds_diff_mean, color='g', label='Difference')
    ax[0].vlines(x,
                 ymin=np.min(psds_diff_mean), ymax=np.max(psds_diff_mean)+5, colors='m',
                 label=('Suggested frequency bounds for NF: {} Hz and {} Hz').format(round(x[0]), round(x[1])))

    ax[0].legend()
    ax[0].set(title='Welch PSD (EEG)', xlabel='Frequency',
              ylabel='Mean Power Spectral Density (dB)', 
              xticks=np.arange(round(np.min(freqs_ec)), round(np.max(freqs_eo)), step=2))

    ## plot topomap
    im, fl = mne.viz.plot_topomap(psds_diff.T[np.where(psds_diff_mean == np.max(psds_diff_mean))[0][0]],
                                  pos=info, 
                                  axes=ax[1], 
                                  vmin=-np.max(psds_diff.T[np.where(psds_diff_mean == np.max(psds_diff_mean))[0][0]]), 
                                  vmax=np.max(psds_diff.T[np.where(psds_diff_mean == np.max(psds_diff_mean))[0][0]])
                                 )
    cbar = fig.colorbar(im, ax=ax[1])
    cbarlabel = ('Power Spectral Density (dB) at {} Hz').format(freqs_eo[np.where(psds_diff_mean == np.max(psds_diff_mean))[0][0]])
    cbar.set_label(cbarlabel)
    fig.tight_layout()
    fig.show()

In [15]:
plot_psd_pb("L:/Lab_MichaelB/Johan/nf/rawdata/T0004/eeg/nk2.eeg", 'L:/Lab_MichaelB/Johan/nf/rawdata/T0004/eeg/nk2.meta', 'L:/Lab_MichaelB/Johan/nf/rawdata/T0004/eeg/nk2.marker')

Creating RawArray with float64 data, n_channels=64, n_times=2584200
    Range : 0 ... 2584199 =      0.000 ...   516.840 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=2584200
    Range : 0 ... 2584199 =      0.000 ...   516.840 secs
Ready.
4 events found
Event IDs: [201 202 203 204]
Applying average reference.
Applying a custom EEG reference.
Effective window size : 3.277 (s)
Effective window size : 3.277 (s)
